In [1]:
#!/usr/bin/env python
import bottle as bt
import returnData

##add these below to new files
import urllib2  
import json  
import sqlite3
import shutil
import os
import yaml
##
import ruamel.yaml
from ruamel.yaml.util import load_yaml_guess_indent

##saving stdout to variable
from cStringIO import StringIO
import sys

from collections import namedtuple
from ansible.parsing.dataloader import DataLoader
from ansible.vars.manager import VariableManager
from ansible.inventory.manager import InventoryManager
from ansible.playbook.play import Play
from ansible.executor.task_queue_manager import TaskQueueManager
from ansible.plugins.callback import CallbackBase
from ansible.executor.playbook_executor import PlaybookExecutor
import ansible.constants as C

class ResultCallback(CallbackBase):
    """A sample callback plugin used for performing an action as results come in

    If you want to collect all results into a single object for processing at
    the end of the execution, look into utilizing the ``json`` callback plugin
    or writing your own custom callback plugin
    """
    def __init__(self):
        super(ResultCallback, self).__init__()
        self.results = []
        
    def v2_runner_on_ok(self, result, **kwargs):
        """Print a json representation of the result

        This method could store the result in an instance attribute for retrieval later
        """
        self.results.append(result)
        print "hi"
        host = result._host
        print(json.dumps({host.name: result._result}, indent=4))
#This file should be located in /YourApp/py/
#Therefore, the file location should be /YourApp/py/runWebAppServer.py
#Do not forget to chmod 755 this file before running on your linux server

def run_playbook(ansible_items):
    print '0'
    #print pb_vars
    #old_stdout = sys.stdout
    #sys.stdout = mystdout = StringIO()
    #print '1'
    playbook_name = ansible_items[0]
    archive_path = 'playbooks/'
    playbook_folder = playbook_name+'/'
    loader = DataLoader()
    inventory_file_path = archive_path+playbook_folder+'inventory'
    #print inventory_file_path
    callback = ResultCallback()
    
    #put file path of inventory file inside sources
    inventory = InventoryManager(loader=loader, sources=[inventory_file_path])
    try:
        if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yml'):
            playbook_path = archive_path + playbook_folder + playbook_name+'.yaml'
            print "Playbook file is using .yaml"
    except:
        print "hi"       
    try:
        if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yaml'):
            playbook_path = archive_path + playbook_folder + playbook_name+'.yml'
            print "Playbook file is using .yml"
    except:
        print "hey"
    variable_manager = VariableManager(loader=loader, inventory=inventory)
    #print '2'
    #print out hosts under type 'dns'
    if not os.path.exists(inventory_file_path):
        print '[INFO] The inventory does not exist'
        sys.exit()
    #print(inventory.get_groups_dict()['dns'])
    if not os.path.exists(playbook_path):
        print '[INFO] The playbook does not exist'
        sys.exit()
    #had to add 'diff' into named tuple & diff=false into Options
    Options = namedtuple('Options', ['listtags', 'listtasks', 'listhosts', 'syntax', 'connection','module_path', 'forks', 'remote_user', 'private_key_file', 'ssh_common_args', 'ssh_extra_args', 'sftp_extra_args', 'scp_extra_args', 'become', 'become_method', 'become_user', 'verbosity', 'check', 'diff'])
    options = Options(listtags=False, listtasks=False, listhosts=False, syntax=False, connection='paramiko', module_path=None, forks=100, remote_user='tbo2', private_key_file=None, ssh_common_args=None, ssh_extra_args=None, sftp_extra_args=None, scp_extra_args=None, become=None, become_method=None, become_user=None, verbosity=None, check=False, diff=False)

    #defining playbook variables in variable manager
    vars_dict = {}
    if playbook_name == 'ping':
        print('trying')
        #vars_dict['ip'] = pb_vars
        #print('printing vars dict: '+ vars_dict)
        variable_manager.extra_vars = {'ip':ansible_items[1]} # This can accomodate various other command line arguments.
    #vars_dict.update(pb_vars)
    #print vars_dict
    elif len(ansible_items) > 1:
        x = extra_variables(ansible_items[1])
        variable_manager.extra_vars = x
    
    passwords = {}
    pbex = PlaybookExecutor(playbooks=[playbook_path], inventory=inventory, variable_manager=variable_manager, loader=loader, options=options, passwords=passwords)
    #pbex._tqm._stdout_callback = callback
    #print pbex._tqm._stdout_callback
    result = pbex.run()
    
    #sys.stdout = old_stdout
    #string = mystdout.getvalue()
    
    
    #d = string.split('TASK [debug] *******************************************************************')[-1].split('{', 1)[-1].split('PLAY RECAP *********************************************************************')[0].strip()
    #d = '{'+d

    #import ast
    #string = ast.literal_eval(d)

    #p = '```'+string+'```'
    return result
    #return string['msg']
def extra_variables(pb_vars):
    vars_dict = {}
    pb_list = pb_vars.split(';')
    for variable in pb_list:
        item = variable.split(':')
        key = str(item[0]).strip()
        value = str(item[1]).strip()
        vars_dict[key] = value
    print vars_dict
    return vars_dict

In [14]:
string = run_playbook(["createtenant"])
print type(string)

0
Playbook file is using .yml

PLAY [ENSURE TENANT CONFIGURATION EXISTS] **************************************

TASK [CREATE TENANT] ***********************************************************
ok: [sandboxapicdc.cisco.com]

TASK [debug] *******************************************************************
ok: [sandboxapicdc.cisco.com] => {
    "msg": {
        "changed": false, 
        "current": [
            {
                "fvTenant": {
                    "attributes": {
                        "descr": "Tenant Created Using Ansible", 
                        "dn": "uni/tn-default", 
                        "name": "default", 
                        "nameAlias": "", 
                        "ownerKey": "", 
                        "ownerTag": ""
                    }
                }
            }
        ], 
        "failed": false
    }
}

PLAY RECAP *********************************************************************
sandboxapicdc.cisco.com    : ok=2    changed=0    unreach

In [ ]:
print string

In [ ]:
x = string
walk_dict_new(x)


In [ ]:
def walk_dict_new(d,depth=0):
    s = ''
    for k,v in sorted(d,key=lambda x: x[0]):
        if isinstance(v, dict):
            if depth > 0:
                x = '\n' + ("  ")*depth + ("%s" % k)
            else:
                x = '\n' + ("  ")*depth + ("%s" % k)
            
            print x
            s+=x
            walk_dict(v,depth+1)
        else:
            if depth > 0:
                y = '\n' +("  ")*depth + "%s %s" % (k, v)
            else:
                y = '\n' + ("  ")*depth + " %s %s" % (k, v)
            s+=y
            print y
    return s


In [ ]:
print s

In [ ]:
def walk_dict_editted(d,depth=0):
    for k,v in sorted(d.items(),key=lambda x: x[0]):
        if isinstance(v, dict):
            if depth > 0:
                x = '\n -' + ("  ")*depth + ("%s" % k)
            else:
                x = ("  ")*depth + ("%s" % k)
            print x
            walk_dict(v,depth+1)
        else:
            if depth > 0:
                y = '\n -' +("  ")*depth + "%s %s" % (k, v)
            else:
                y = ("  ")*depth + "%s %s" % (k, v)
            print y 


In [ ]:

p = 'https://www.youtube.com/watch?v='+string['msg']['items'][0]['id']['videoId']
print type(p)
string = '```'+p+'````'
print string

In [ ]:
eeee = string.split('TASK [debug] *******************************************************************')[-1].strip
print eeee

In [ ]:
eeee = eeee.split('{', 1)[-1]
print eeee

In [ ]:
eeee = eeee.split('PLAY RECAP *********************************************************************')[0]
d = eeee.strip()
d = '{'+d
#d = json.load(d)
import ast
d = ast.literal_eval(d)
p = 'https://www.youtube.com/watch?v='+d['msg']['items'][0]['id']['videoId']
string = '```'+str(p)+'````'

In [ ]:
playbook_name = 'umbrella'
archive_path = 'playbooks/'
playbook_folder = playbook_name+'/'
try:
    if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yml'):
        playbook_path = archive_path + playbook_folder + playbook_name+'.yaml'
        print "Playbook file is using .yaml"
except:
    print "hi"       
try:
    if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yaml'):
        playbook_path = archive_path + playbook_folder + playbook_name+'.yml'
        print "Playbook file is using .yml"
except:
    print "hey"
with open(playbook_path, 'r') as f:
    doc = yaml.load(f)
    yaml = doc[0]
print yaml['vars']

In [ ]:
print type(info_dict)

In [ ]:

print "got here"
with open(playbook_path) as d:
    print "next"
    data = yaml.load(d)
    print "okay"
print doc
yaml = doc[0]
yaml = yaml['vars']
formatted_yaml_vars = "The variables you can modify are:"
for key, value in yaml.iteritems():
    formatted_yaml_vars+= "\n- "+str(key)+" : "+str(value)
print formatted_yaml_vars

In [ ]:
playbook_name = 'umbrella'
archive_path = 'playbooks/'
playbook_folder = playbook_name+'/'
try:
    if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yml'):
            playbook_path = archive_path + playbook_folder + playbook_name+'.yaml'
            print "Playbook file is using .yaml"
except:
    print "hi"       
try:
    if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yaml'):
        playbook_path = archive_path + playbook_folder + playbook_name+'.yml'
        print "Playbook file is using .yml"
except:
    print "hey"
config, ind, bsi = load_yaml_guess_indent(open(playbook_path))
print(type(config))



In [ ]:
print(config[0])

In [ ]:
yaml = config[0]

In [ ]:
print yaml['vars']

In [ ]:
print len(yaml)

In [ ]:
formatted_yaml_vars = "The variables you can modify are:"
yaml = yaml['vars']
for key, value in yaml.iteritems():
    formatted_yaml_vars+= "\n- "+str(key)+" : "+str(value)
print formatted_yaml_vars

In [ ]:
def get_vars_playbook(filename):
    print filename
    playbook_name = filename.replace(" ", "")
    print playbook_name+"1"
    archive_path = 'playbooks/'
    playbook_folder = playbook_name+'/'
    try:
        if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yml'):
            playbook_path = archive_path + playbook_folder + playbook_name+'.yaml'
            print "Playbook file is using .yaml"
    except:
        print "hi"       
    try:
        if not os.path.exists(archive_path + playbook_folder + playbook_name+'.yaml'):
            playbook_path = archive_path + playbook_folder + playbook_name+'.yml'
            print "Playbook file is using .yml"
    except:
        print "hey"
    print "got here get variables"
    print playbook_path
    config, ind, bsi = load_yaml_guess_indent(open(playbook_path))
    yaml = config[0]
    print "next part"
    print yaml
    formatted_yaml_vars = "The variables you can modify are:"
    yaml = yaml['vars']
    for key, value in yaml.iteritems():
        formatted_yaml_vars+= "\n- "+str(key)+" : "+str(value)
    return formatted_yaml_vars

In [ ]:
get_vars_playbook('umbrella')